In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import Image
from tqdm import tqdm
from scipy import stats

In [2]:
device = torch.device('cuda:0')

In [3]:
# data_transforms = {
#     'train': transforms.Compose([
# #         transforms.Resize((310, 310)),
#         transforms.Resize((224, 224)),
# #         transforms.CenterCrop(224),
# #         transforms.RandomResizedCrop(224),
# #         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
#     'val': transforms.Compose([
# #         transforms.Resize((310, 310)),
#         transforms.Resize((224, 224)),
# #         transforms.Resize(256),
# #         transforms.CenterCrop(224),
# #         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
# }
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [4]:
class ImageDataset(Dataset):
    def __init__(self, paths, labels, transform=None, stride = 1, max_size=None):
        self.paths = paths[::stride]
        if max_size is not None:
            self.paths = self.paths[:max_size]
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img_name = self.paths[idx]
        img = Image.open(img_name).convert('RGB')
        
        img_tensor = self.transform(img)

        return img_tensor, self.labels[idx]

In [5]:
train_file = 'commercials/data/train.txt'
val_file = 'commercials/data/val.txt'
test_file = 'commercials/data/test.txt'

In [6]:
def read_file(filename):
    paths = []
    labels = []
    with open(filename, 'r') as f:
        for line in f.readlines():
            video, image, label = line.split(' ')
            paths.append('commercials/images/{}/{:04d}.jpg'.format(video, int(image)))
            labels.append(int(label))
            
    return paths, labels

In [7]:
train_paths, Y_train = read_file(train_file)
val_paths, Y_val = read_file(val_file)
test_paths, Y_test = read_file(test_file)

In [8]:
image_datasets = {
    'train': ImageDataset(train_paths, Y_train, transform = data_transforms['train'],
                          stride=1, max_size=None),
    'val_train': ImageDataset(val_paths, Y_val, transform = data_transforms['train'],
                       stride=1, max_size=None),
    'val': ImageDataset(val_paths, Y_val, transform = data_transforms['val'],
                       stride=1, max_size=None),
    'test': ImageDataset(test_paths, Y_test, transform = data_transforms['val'],
                        stride=1, max_size=None)
}

In [9]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x],
                                              batch_size=4,
                                              shuffle='train' in x,
                                              num_workers=4,
                                              pin_memory=True)
                                              for x in image_datasets}

In [10]:
dataset_sizes = {
    x: len(image_datasets[x])
    for x in image_datasets
}

In [11]:
dataset_sizes

{'train': 64130, 'val_train': 9479, 'val': 9479, 'test': 7496}

In [12]:
def safe_divide(a, b):
    return a / b if b > 0 else 0

In [13]:
def train_model(model, criterion, optimizer, scheduler, train_dl, val_dl, test_dl=None,
                num_epochs=25, return_best=False, verbose=True, log_file=None):
    print(train_dl, val_dl, test_dl)
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_epoch = 0
    best_test_acc = 0.0
    best_f1 = 0.0
    best_precision = 0.0
    best_recall = 0.0
    
    phases = ['train', 'val', 'test'] if test_dl is not None else ['train', 'val']

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in phases:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
                dl = dataloaders[train_dl]
                dataset_size = dataset_sizes[train_dl]
                
            elif phase == 'val':
                model.eval()   # Set model to evaluate mode
                dl = dataloaders[val_dl]
                dataset_size = dataset_sizes[val_dl]
            else:
                model.eval()
                dl = dataloaders[test_dl]
                dataset_size = dataset_sizes[test_dl]

            running_loss = 0.0
            running_corrects = 0
            true_positives = 0.
            true_negatives = 0.
            false_positives = 0.
            false_negatives = 0.

            # Iterate over data.
            for inputs, labels in dl:
                inputs = inputs.to(device)
                labels = labels.to(device).float()

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    preds = torch.where(
                        outputs >= 0.,
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                    target = torch.where(
                        labels >= 0.5,
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device)
                    )
                    loss = criterion(outputs.view(target.shape), target)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                label_vals = torch.where(
                    labels >= 0.5,
                    torch.tensor([1.]).to(device),
                    torch.tensor([0.]).to(device)
                )
                correct = preds.view(label_vals.shape) == label_vals.data
                running_corrects += torch.sum(correct)
                
                true_positives += torch.sum(
                    torch.where(
                        (correct == 1.) * (label_vals == 1.),
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                )
                true_negatives += torch.sum(
                    torch.where(
                        (correct == 1.) * (label_vals == 0.),
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                )
                false_positives += torch.sum(
                    torch.where(
                        (correct == 0.) * (label_vals == 0.),
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                )
                false_negatives += torch.sum(
                    torch.where(
                        (correct == 0.) * (label_vals == 1.),
                        torch.tensor([1.]).to(device),
                        torch.tensor([0.]).to(device))
                )
            
            epoch_loss = running_loss / dataset_size
            epoch_acc = running_corrects.double() / dataset_size
            epoch_pre = safe_divide(true_positives, (true_positives + false_positives))
            epoch_recall = safe_divide(true_positives, (true_positives + false_negatives))
            epoch_f1 = safe_divide(2 * epoch_pre * epoch_recall, (epoch_pre + epoch_recall))

            if verbose:
                print('{} Loss: {:.4f} Acc: {:.4f} Pre: {:.4f} Rec: {:.4f} F1: {:.4f}'.format(
                    phase, epoch_loss, epoch_acc, epoch_pre, epoch_recall, epoch_f1))
                print('TP: {} TN: {} FP: {} FN: {}'.format(
                    true_positives.data, true_negatives.data, false_positives.data, false_negatives.data))
            if log_file is not None:
                log_file.write('Phase: {0}\t'
                               'Epoch: [{1}/{2}]\t'
                               'Loss: {loss_c:.4f}\t'
                               'Acc: {acc:.4f}\t'
                               'Pre: {pre:.4f}\t'
                               'Rec: {rec:.4f}\t'
                               'F1: {f1:.4f}\t'
                               'TP: {tp} '
                               'TN: {tn} '
                               'FP: {fp} '
                               'FN: {fn}\n'.format(
                                   phase, epoch + 1, num_epochs, loss_c=epoch_loss,
                                   acc=epoch_acc, pre=epoch_pre, rec=epoch_recall,
                                   f1=epoch_f1, tp=int(true_positives.data), tn=int(true_negatives.data),
                                   fp=int(false_positives.data), fn=int(false_negatives.data)
                               ))
                log_file.flush()

            # deep copy the model
            if phase == 'val' and epoch_f1 > best_f1:
                best_acc = epoch_acc
                best_f1 = epoch_f1
                best_precision = epoch_pre
                best_recall = epoch_recall
                best_epoch = epoch
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'test' and best_epoch == epoch:
                best_test_acc = epoch_acc

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    
    if return_best:
        print('Best epoch: {}'.format(best_epoch))
        print('Best val Acc: {:4f}'.format(best_acc))
        print('Best val Pre: {:4f}'.format(best_precision))
        print('Best val Rec: {:4f}'.format(best_recall))
        print('Best val F1: {:4f}'.format(best_f1))
        print('Test Acc: {:4f}'.format(best_test_acc))

        # load best model weights
        model.load_state_dict(best_model_wts)
    return model

In [15]:
path = 'models/transfer_learning'
for seed in range(1):
    torch.manual_seed(seed)
    model_ts = models.resnet18(pretrained=True)
    num_ftrs = model_ts.fc.in_features
    model_ts.fc = nn.Linear(num_ftrs, 1)

    model_ts = model_ts.to(device)

#     criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([10.]).to(device))
#     criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([5.]).to(device))
    
    criterion = nn.BCEWithLogitsLoss()
    
    # Observe that all parameters are being optimized
    optimizer_ts = optim.SGD(model_ts.parameters(), lr=0.001, momentum=0.9)

    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler_ts = lr_scheduler.StepLR(optimizer_ts, step_size=7, gamma=0.1)
    
    if not os.path.exists(path):
        os.makedirs(path)
    with open(os.path.join(path, 'seed_{}.log'.format(seed)), 'w') as log_file:
        model_ts = train_model(model_ts, criterion, optimizer_ts, exp_lr_scheduler_ts,
                               'val_train', 'val', test_dl='test', num_epochs=25, verbose=True,
                               log_file=log_file, return_best=True)
        torch.save(model_ts.state_dict(), os.path.join(path, 'seed_{}.pth'.format(seed)))

val_train val test
Epoch 0/24
----------
train Loss: 0.3653 Acc: 0.8468 Pre: 0.7424 Rec: 0.7252 F1: 0.7337
TP: 2000.0 TN: 6027.0 FP: 694.0 FN: 758.0
val Loss: 0.1547 Acc: 0.9543 Pre: 0.9667 Rec: 0.8731 F1: 0.9175
TP: 2408.0 TN: 6638.0 FP: 83.0 FN: 350.0
test Loss: 0.2187 Acc: 0.9288 Pre: 0.9192 Rec: 0.8494 F1: 0.8829
TP: 2014.0 TN: 4948.0 FP: 177.0 FN: 357.0

Epoch 1/24
----------
train Loss: 0.2947 Acc: 0.8889 Pre: 0.8046 Rec: 0.8165 F1: 0.8105
TP: 2252.0 TN: 6174.0 FP: 547.0 FN: 506.0
val Loss: 0.1567 Acc: 0.9628 Pre: 0.9804 Rec: 0.8898 F1: 0.9329
TP: 2454.0 TN: 6672.0 FP: 49.0 FN: 304.0
test Loss: 0.2497 Acc: 0.9320 Pre: 0.9243 Rec: 0.8549 F1: 0.8883
TP: 2027.0 TN: 4959.0 FP: 166.0 FN: 344.0

Epoch 2/24
----------
train Loss: 0.2632 Acc: 0.9004 Pre: 0.8263 Rec: 0.8328 F1: 0.8295
TP: 2297.0 TN: 6238.0 FP: 483.0 FN: 461.0
val Loss: 0.1042 Acc: 0.9748 Pre: 0.9691 Rec: 0.9434 F1: 0.9561
TP: 2602.0 TN: 6638.0 FP: 83.0 FN: 156.0
test Loss: 0.2490 Acc: 0.9141 Pre: 0.8328 Rec: 0.9114 F1: 0.

train Loss: 0.1463 Acc: 0.9429 Pre: 0.8980 Rec: 0.9068 F1: 0.9024
TP: 2501.0 TN: 6437.0 FP: 284.0 FN: 257.0
val Loss: 0.0571 Acc: 0.9822 Pre: 0.9672 Rec: 0.9717 F1: 0.9694
TP: 2680.0 TN: 6630.0 FP: 91.0 FN: 78.0
test Loss: 0.1858 Acc: 0.9366 Pre: 0.8674 Rec: 0.9439 F1: 0.9041
TP: 2238.0 TN: 4783.0 FP: 342.0 FN: 133.0

Training complete in 51m 27s
Best epoch: 22
Best val Acc: 0.983859
Best val Pre: 0.975886
Best val Rec: 0.968455
Best val F1: 0.972156
Test Acc: 0.942636


# Understand Mistakes

In [56]:
path = 'models/traditional_supervision_no_augmentation'
model_ts = models.resnet18(pretrained=True)
num_ftrs = model_ts.fc.in_features
model_ts.fc = nn.Linear(num_ftrs, 1)
model_ts.load_state_dict(torch.load(
    os.path.join(path, 'seed_0.pth')
))
model_ts.to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([3.]).to(device))

In [57]:
model = model_ts.eval()   # Set model to evaluate mode
dl = dataloaders['val']
dataset_size = dataset_sizes['val']

In [58]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.cpu().numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(str(title))
    plt.pause(0.001)  # pause a bit so that plots are updated

In [59]:
def pos_negs(predictions, gt):
    correct = np.where(np.array(predictions) == np.array(gt), 1, 0)
    incorrect = np.where(np.array(predictions) == np.array(gt), 0, 1)
    
    tp = np.where(correct * np.where(predictions == np.array(1), 1, 0), 1, 0)
    tn = np.where(correct * np.where(predictions == np.array(0), 1, 0), 1, 0)
    fp = np.where(incorrect * np.where(predictions == np.array(1), 1, 0), 1, 0)
    fn = np.where(incorrect * np.where(predictions == np.array(0), 1, 0), 1, 0)
    
    return tp, tn, fp, fn

def acc_prf1(predictions, gt):
    tp, tn, fp, fn = pos_negs(predictions, gt)
    
    acc = (np.sum(tp) + np.sum(tn)) / (np.sum(tp) + np.sum(tn) + np.sum(fp) + np.sum(fn))
    precision = np.sum(tp) / (np.sum(tp) + np.sum(fp))
    recall = np.sum(tp) / (np.sum(tp) + np.sum(fn))
    f1 = 2 * precision * recall / (precision + recall)
    
    return acc, precision, recall, f1, np.sum(tp), np.sum(tn), np.sum(fp), np.sum(fn)

def smooth_predictions(preds, window_radius = 3):
    result = []
    for i in range(len(preds)):
        start = max(0, i - window_radius)
        end = min(len(preds), i + window_radius)
        window = preds[start:end]
        result += [max(window, key=window.count)]
    
    return result

In [60]:
running_corrects = 0
true_positives = 0.
true_negatives = 0.
false_positives = 0.
false_negatives = 0.

i = 0

predictions = []
gt_labels = []
# Iterate over data.
for inputs, labels in dl:
    inputs = inputs.to(device)
    labels = labels.to(device).float()

    # forward
    # track history if only in train
    with torch.set_grad_enabled(False):
        outputs = model(inputs)
        preds = torch.where(
            outputs >= 0.,
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
        target = torch.where(
            labels >= 0.5,
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device)
        )
        
    predictions += preds.cpu().numpy().tolist()
    gt_labels += labels.cpu().numpy().tolist()

    # statistics
    label_vals = torch.where(
        labels >= 0.5,
        torch.tensor([1.]).to(device),
        torch.tensor([0.]).to(device)
    )
    correct = preds.view(label_vals.shape) == label_vals.data
    running_corrects += torch.sum(correct)

    true_positives += torch.sum(
        torch.where(
            (correct == 1.) * (label_vals == 1.),
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
    )
    true_negatives += torch.sum(
        torch.where(
            (correct == 1.) * (label_vals == 0.),
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
    )
    false_positives += torch.sum(
        torch.where(
            (correct == 0.) * (label_vals == 0.),
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
    )
    false_negatives += torch.sum(
        torch.where(
            (correct == 0.) * (label_vals == 1.),
            torch.tensor([1.]).to(device),
            torch.tensor([0.]).to(device))
    )
    
    num_fp = torch.sum(
    torch.where(
        (correct == 0.) * (label_vals == 0.),
        torch.tensor([1.]).to(device),
        torch.tensor([0.]).to(device))
    )

#     if num_fp > 0:
#         print(num_fp)
#         print(torch.where(
#             (correct == 0.) * (label_vals == 0.),
#             torch.tensor([1.]).to(device),
#             torch.tensor([0.]).to(device)))
#         print(i)
#         out = torchvision.utils.make_grid(inputs)
# #         imshow(out, title=preds.cpu().numpy().tolist())
#         imshow(out)
    
    i += 1

epoch_acc = running_corrects.double() / dataset_size
epoch_pre = safe_divide(true_positives, (true_positives + false_positives))
epoch_recall = safe_divide(true_positives, (true_positives + false_negatives))
epoch_f1 = safe_divide(2 * epoch_pre * epoch_recall, (epoch_pre + epoch_recall))

print('Acc: {:.4f} Pre: {:.4f} Rec: {:.4f} F1: {:.4f}'.format(
    epoch_acc, epoch_pre, epoch_recall, epoch_f1))
print('TP: {} TN: {} FP: {} FN: {}'.format(
    true_positives.data, true_negatives.data, false_positives.data, false_negatives.data))

predictions = [p[0] for p in predictions]

smoothed_preds = smooth_predictions(predictions, 3)

print("Smoothed stats:")
print(acc_prf1(smoothed_preds, gt_labels))

Acc: 0.9787 Pre: 0.9587 Rec: 0.9685 F1: 0.9636
TP: 2671.0 TN: 6606.0 FP: 115.0 FN: 87.0
Smoothed stats:
(0.9592784048950311, 0.93033381712627, 0.9296591733139956, 0.9299963728690607, 2564, 6529, 192, 194)
